# TRAINING I3W


# A) Create Envorinment, Vehicles etc

### General Parameter

In [1]:
# Define horizon as a variable to ensure consistent use across notebook (length of one rollout)
HORIZON=500                                 #103 max Horizon, wenn es vor verlassen abbrechen soll!, default war 500

# name of the experiment
experiment_name = "IntersectionExample"

# scenario class
import flow.scenarios as scenarios
print("Available scenarios:")
print(scenarios.__all__)
scenario_name = "IntersectionTWScenario"

# environment class
import flow.multiagent_envs as flowenvs
print("\nAvailable environments:")
print(flowenvs.__all__)
env_name = "MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit"

Available scenarios:
['Scenario', 'BayBridgeScenario', 'BayBridgeTollScenario', 'BottleneckScenario', 'Figure8Scenario', 'SimpleGridScenario', 'HighwayScenario', 'LoopScenario', 'MergeScenario', 'TwoLoopsOneMergingScenario', 'MultiLoopScenario', 'IntersectionScenarioTW']

Available environments:
['MultiEnv', 'MultiAgentAccelEnv', 'MultiWaveAttenuationPOEnv', 'MultiAgentIntersectionEnv', 'MultiAgentTeamSpiritIntersectionEnv', 'MultiAgentIntersectionEnv_baseline_1', 'MultiAgentIntersectionEnv_baseline_2', 'MultiAgentIntersectionEnv_baseline_3', 'MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit']


### Net Parameter

In [2]:
from flow.core.params import NetParams
from flow.scenarios.intersection import ADDITIONAL_NET_PARAMS

additionalNetParams = {
            "edge_length": 40,
            "lanes": 1,
            "speed_limit": 30
        }

net_params = NetParams( no_internal_links=False,                  #default: True   !! damit Kreuzungen nicht überspr. werden
                        inflows=None,                             #default: None
                        osm_path=None,                            #default: None
                        netfile=None,                             #default: None
                        additional_params=additionalNetParams     #default: None   !!
                      )

### InitialConfig Parameter

In [3]:
from flow.core.params import InitialConfig

initial_config = InitialConfig( shuffle=True,                            #default: False         !!
                                spacing="custom",                        #default: "uniform"     !!
                                min_gap=10,                              #default: 0
                                perturbation=29.99,                      #default: 0.0            !!        
                                x0=0,                                    #default: 0
                                bunching=0,                              #default: 0
                                lanes_distribution=float("inf"),         #default: float("inf")
                                edges_distribution="all",                #default: "all"
                                additional_params=None )                 #default: None

### SUMO Parameter

In [4]:
from flow.core.params import SumoParams

sumo_params = SumoParams( port = None,                  #default: None
                          sim_step=0.1,                 #default: 0.1
                          emission_path=None,           #default: None
                          lateral_resolution=None,      #default: None
                          no_step_log=True,             #default: True
                          render=False,                 #default: False
                          save_render=False,            #default: False
                          sight_radius=25,              #default: 25
                          show_radius=False,            #default: False
                          pxpm=2,                       #default: 2
                          overtake_right=False,         #default: False    
                          seed=None,                    #default: None
                          restart_instance=False,       #default: False
                          print_warnings=True,          #default: True
                          teleport_time=-1,             #default: -1
                          num_clients=1,                #default: 1
                          sumo_binary=None )            #default: None

### Environment Parameter

In [5]:
from flow.core.params import EnvParams

additionalEnvParams = {
        # maximum acceleration of autonomous vehicles
        "max_accel": 3,
        # maximum deceleration of autonomous vehicles
        "max_decel": 3,
        # desired velocity for all vehicles in the network, in m/s
        "target_velocity": 30,
        # initial teamspirit
        "ap_teamspirit_0": -1,
        "ap_teamspirit_1": -1,
        # shuffle teamspirit?
        "ap_teamspirit_shuffle": True    
    }

env_params = EnvParams( additional_params=additionalEnvParams, #default: None    !!
                        horizon=HORIZON,                       #default: 500     !!
                        warmup_steps=0,                        #default: 0       
                        sims_per_step=1,                       #default: 1
                        evaluate=False )                       #default: False

### Vehicles Parameter

In [6]:
from flow.core.params import VehicleParams

# import vehicles dynamics models
#from flow.controllers import SumoCarFollowingController
from flow.controllers import ContinuousRouter
#from flow.controllers.lane_change_controllers import SumoLaneChangeController
from flow.controllers.lane_change_controllers import StaticLaneChanger
from flow.controllers import RLController
from flow.core.params import SumoLaneChangeParams
from flow.core.params import SumoCarFollowingParams
from random import *

vehicles = VehicleParams()

#### Add RL-Agent controlled vehicles 

In [7]:
# car following parameters, default: None
cf_parameter = SumoCarFollowingParams(
                speed_mode="aggressive")
# lane change parameters, default: None
lc_parameter =  None

vehicles.add( # name of the vehicle
                veh_id = "rl",
              # acceleration controller, default: (SumoCarFollowingController, {})
                acceleration_controller=(RLController, {}),
              # lane_change_controller, default: (SumoLaneChangeController, {})
                lane_change_controller=(StaticLaneChanger,{}),
              # routing controller, default: None
                routing_controller=(ContinuousRouter, {}),
              # initial speed, default: 0
                initial_speed=0,
              # number of vehicles, default: 1 
                num_vehicles=2,
                
                car_following_params=cf_parameter
              # speed mode, default: "right_of_way"
                #speed_mode="aggressive",
              # lane change mode, default: "no_lat_collide"
                #lane_change_mode="aggressive", 
              # car following parameter, default: None
                #sumo_car_following_params=cf_parameter,
              # lane change parameter, default: None
                #sumo_lc_params=lc_parameter
)

### Flow Parameter

In [8]:
# Creating flow_params. Make sure the dictionary keys are as specified. 
flow_params = dict( # name of the experiment
                      exp_tag=experiment_name,
                    # name of the flow environment the experiment is running on
                      env_name=env_name,
                    # name of the scenario class the experiment uses
                      scenario=scenario_name,
                    # simulator that is used by the experiment
                      simulator='traci',
                    # sumo-related parameters (see flow.core.params.SumoParams)
                      sim=sumo_params,
                    # environment related parameters (see flow.core.params.EnvParams)
                      env=env_params,
                    # network-related parameters (see flow.core.params.NetParams and
                    # the scenario's documentation or ADDITIONAL_NET_PARAMS component)
                      net=net_params,
                    # vehicles to be placed in the network at the start of a rollout 
                    # (see flow.core.vehicles.Vehicles)
                      veh=vehicles,
                   # (optional) parameters affecting the positioning of vehicles upon 
                   # initialization/reset (see flow.core.params.InitialConfig)
                      initial=initial_config
                )

# B) Training

In [9]:
import json

import ray
try:
    from ray.rllib.agents.agent import get_agent_class
except ImportError:
    from ray.rllib.agents.registry import get_agent_class
from ray.tune import run_experiments
from ray.tune.registry import register_env

from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder

from ray import tune
from ray.rllib.agents.ppo.ppo_policy_graph import PPOPolicyGraph

In [10]:
# number of parallel workers
N_CPUS = 2
# number of rollouts per training iteration
N_ROLLOUTS = 20

ray.init(redirect_output=True, num_cpus=N_CPUS+1)

Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-03-30_18-08-52_7633/logs.
Waiting for redis server at 127.0.0.1:42231 to respond...
Waiting for redis server at 127.0.0.1:50969 to respond...
Starting the Plasma object store with 6.554658406 GB memory using /dev/shm.

View the web UI at http://localhost:8889/notebooks/ray_ui.ipynb?token=e780c08def7cacb64256967715c94ae4cfd2fb06f11817b8



{'node_ip_address': '192.168.2.102',
 'object_store_addresses': ['/tmp/ray/session_2019-03-30_18-08-52_7633/sockets/plasma_store'],
 'raylet_socket_names': ['/tmp/ray/session_2019-03-30_18-08-52_7633/sockets/raylet'],
 'redis_address': '192.168.2.102:42231',
 'webui_url': 'http://localhost:8889/notebooks/ray_ui.ipynb?token=e780c08def7cacb64256967715c94ae4cfd2fb06f11817b8'}

In [11]:
# The algorithm or model to train. This may refer to "
#      "the name of a built-on algorithm (e.g. RLLib's DQN "
#      "or PPO), or a user-defined trainable function or "
#      "class registered in the tune registry.")
alg_run = "PPO"

agent_cls = get_agent_class(alg_run)
config = agent_cls._default_config.copy()
config["num_workers"] = N_CPUS  # number of parallel workers
config["train_batch_size"] = HORIZON * N_ROLLOUTS  # batch size
config["gamma"] = 0.999  # discount rate default 0.999
config["model"].update({"fcnet_hiddens": [100, 50, 25]})  # size of hidden layers in network defaule 64 32
config["use_gae"] = True  # using generalized advantage estimation
config["lambda"] = 0.97  
#config["sgd_minibatch_size"] = min(16 * 1024, config["train_batch_size"])  # stochastic gradient descent
#config["sample_batch_size"] = config["train_batch_size"]/config["num_workers"] # 200 default, trotzdem zu hoch?
config["kl_target"] = 0.02  # target KL divergence
config["num_sgd_iter"] = 10  # number of SGD iterations
config["horizon"] = HORIZON  # rollout horizon

# save the flow params for replay
flow_json = json.dumps(flow_params, cls=FlowParamsEncoder, sort_keys=True,
                       indent=4)  # generating a string version of flow_params
config['env_config']['flow_params'] = flow_json  # adding the flow_params to config dict
config['env_config']['run'] = alg_run

# Call the utility function make_create_env to be able to 
# register the Flow env for this experiment
create_env, gym_name = make_create_env(params=flow_params, version=0)

# Register as rllib env with Gym
register_env(gym_name, create_env)

In [12]:
# multi agent policy mapping
test_env = create_env()
obs_space = test_env.observation_space
act_space = test_env.action_space

def gen_policy():
    return (PPOPolicyGraph, obs_space, act_space, {})

# Setup PG with an ensemble of `num_policies` different policy graphs
policy_graphs = {'rl_0': gen_policy()}
    
def policy_mapping_fn(agent_id):
    return 'rl_0'

config.update({
        'multiagent': {
            'policy_graphs': policy_graphs,
            'policy_mapping_fn': tune.function(policy_mapping_fn),
            'policies_to_train': ['rl_0']
        }
    })

 Starting SUMO on port 55245


New Teamspirit:
-0.5393521226962619
0.17788004549039993


In [13]:
trials = run_experiments({
    flow_params["exp_tag"]: {
        "run": alg_run,  # RL algorithm to run
        "env": gym_name,  # environment name generated earlier
        "config": {  # configuration params (must match "run" value)
            **config
        },
        "checkpoint_freq": 1,  # number of iterations between checkpoints
        "max_failures": 999,
        "stop": {  # stopping conditions
            "training_iteration": 1000,  # number of iterations to stop after
        },
    },
})

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/3 CPUs, 0/1 GPUs
Memory usage on this node: 3.7/16.4 GB

Created LogSyncer for /home/thorsten/ray_results/IntersectionExample/PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0_2019-03-30_18-08-54osfmm20k -> 
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 3.8/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0:	RUNNING

Result for PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0:
  custom_metrics: {}
  date: 2019-03-30_18-09-44
  done: false
  episode_len_mean: 450.3333333333333
  episode_reward_max: 446.540044230622
  episode_reward_mean: 237.318160124313
  episode_reward_min: -65.25893908935677
  episodes_this_iter: 21
  episodes_total: 21
  experiment_id: 317c3905198e469ea331d85175f15b15
  hostname: Gandalf
  info:
    grad_time_ms: 4671.

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 4.3/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0:	RUNNING [pid=7681], 121 s, 5 iter, 50000 ts, 483 rew

Result for PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0:
  custom_metrics: {}
  date: 2019-03-30_18-11-46
  done: false
  episode_len_mean: 317.26
  episode_reward_max: 1107.553908557878
  episode_reward_mean: 560.8260890809104
  episode_reward_min: -93.2680168733728
  episodes_this_iter: 35
  episodes_total: 168
  experiment_id: 317c3905198e469ea331d85175f15b15
  hostname: Gandalf
  info:
    grad_time_ms: 4204.283
    load_time_ms: 9.054
    num_steps_sampled: 60000
    num_steps_trained: 60000
    rl_0:
      cur_kl_coeff: 0.006250001490116119
      cur_lr: 4.999999873689376e-05
      entropy: 1.4084738492965698
      kl: 0.0033381700050085783
      polic

Result for PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0:
  custom_metrics: {}
  date: 2019-03-30_18-13-47
  done: false
  episode_len_mean: 197.98
  episode_reward_max: 1124.4251855484695
  episode_reward_mean: 623.4015024460542
  episode_reward_min: -91.32249735106
  episodes_this_iter: 54
  episodes_total: 393
  experiment_id: 317c3905198e469ea331d85175f15b15
  hostname: Gandalf
  info:
    grad_time_ms: 4081.259
    load_time_ms: 1.616
    num_steps_sampled: 110000
    num_steps_trained: 110000
    rl_0:
      cur_kl_coeff: 0.00019531254656612873
      cur_lr: 4.999999873689376e-05
      entropy: 1.372615933418274
      kl: 0.0037068778183311224
      policy_loss: -0.0013310007052496076
      total_loss: 2731.52685546875
      vf_explained_var: 0.09271156042814255
      vf_loss: 2731.528564453125
    sample_time_ms: 20124.788
    update_time_ms: 6.625
  iterations_since_restore: 11
  node_ip: 192.168.2.102
  num_metric_batches_dropped: 0
  pid: 7681
  policy_reward_mea

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 4.4/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0:	RUNNING [pid=7681], 390 s, 16 iter, 160000 ts, 561 rew

Result for PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0:
  custom_metrics: {}
  date: 2019-03-30_18-16-16
  done: false
  episode_len_mean: 163.06
  episode_reward_max: 1093.2206064303496
  episode_reward_mean: 634.8426057010863
  episode_reward_min: -69.90847952706781
  episodes_this_iter: 60
  episodes_total: 724
  experiment_id: 317c3905198e469ea331d85175f15b15
  hostname: Gandalf
  info:
    grad_time_ms: 4065.489
    load_time_ms: 1.515
    num_steps_sampled: 170000
    num_steps_trained: 170000
    rl_0:
      cur_kl_coeff: 3.0517585400957614e-06
      cur_lr: 4.999999873689376e-05
      entropy: 1.3331423997879028
      kl: 0.003791195573285222
    

Result for PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0:
  custom_metrics: {}
  date: 2019-03-30_18-18-19
  done: false
  episode_len_mean: 177.21
  episode_reward_max: 1117.115998055278
  episode_reward_mean: 672.6511593510472
  episode_reward_min: -105.99853480988938
  episodes_this_iter: 59
  episodes_total: 1016
  experiment_id: 317c3905198e469ea331d85175f15b15
  hostname: Gandalf
  info:
    grad_time_ms: 4063.11
    load_time_ms: 1.5
    num_steps_sampled: 220000
    num_steps_trained: 220000
    rl_0:
      cur_kl_coeff: 1.907349087559851e-07
      cur_lr: 4.999999873689376e-05
      entropy: 1.3433706760406494
      kl: 0.004157834220677614
      policy_loss: -0.002163971308618784
      total_loss: 3686.760009765625
      vf_explained_var: 0.28146669268608093
      vf_loss: 3686.761962890625
    sample_time_ms: 20626.799
    update_time_ms: 6.148
  iterations_since_restore: 22
  node_ip: 192.168.2.102
  num_metric_batches_dropped: 0
  pid: 7681
  policy_reward_mea

...

Result for PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0:
  custom_metrics: {}
  date: 2019-03-31_00-37-11
  done: false
  episode_len_mean: 170.69
  episode_reward_max: 1162.9196071294193
  episode_reward_mean: 1008.9673813009562
  episode_reward_min: 53.433188997279416
  episodes_this_iter: 59
  episodes_total: 57081
  experiment_id: 317c3905198e469ea331d85175f15b15
  hostname: Gandalf
  info:
    grad_time_ms: 3883.737
    load_time_ms: 1.603
    num_steps_sampled: 9790000
    num_steps_trained: 9790000
    rl_0:
      cur_kl_coeff: 0.12882612645626068
      cur_lr: 4.999999873689376e-05
      entropy: 0.7407231330871582
      kl: 0.015120946802198887
      policy_loss: -0.003001484554260969
      total_loss: 437.1906433105469
      vf_explained_var: 0.9822737574577332
      vf_loss: 437.19171142578125
    sample_time_ms: 19826.41
    update_time_ms: 6.088
  iterations_since_restore: 979
  node_ip: 192.168.2.102
  num_metric_batches_dropped: 0
  pid: 7681
  policy_rewar

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 7.7/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0:	RUNNING [pid=7681], 23350 s, 984 iter, 9840000 ts, 999 rew

Result for PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0:
  custom_metrics: {}
  date: 2019-03-31_00-39-34
  done: false
  episode_len_mean: 166.95
  episode_reward_max: 1128.7375414523162
  episode_reward_mean: 982.6853723999925
  episode_reward_min: -65.12621023927525
  episodes_this_iter: 58
  episodes_total: 57430
  experiment_id: 317c3905198e469ea331d85175f15b15
  hostname: Gandalf
  info:
    grad_time_ms: 3894.278
    load_time_ms: 1.589
    num_steps_sampled: 9850000
    num_steps_trained: 9850000
    rl_0:
      cur_kl_coeff: 0.12882612645626068
      cur_lr: 4.999999873689376e-05
      entropy: 0.929443359375
      kl: 0.02421778254210949
    

Result for PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0:
  custom_metrics: {}
  date: 2019-03-31_00-41-33
  done: false
  episode_len_mean: 187.21
  episode_reward_max: 1125.3407439612251
  episode_reward_mean: 987.5414882351419
  episode_reward_min: 67.76582654437601
  episodes_this_iter: 49
  episodes_total: 57713
  experiment_id: 317c3905198e469ea331d85175f15b15
  hostname: Gandalf
  info:
    grad_time_ms: 3892.186
    load_time_ms: 1.629
    num_steps_sampled: 9900000
    num_steps_trained: 9900000
    rl_0:
      cur_kl_coeff: 0.12882612645626068
      cur_lr: 4.999999873689376e-05
      entropy: 1.9097716808319092
      kl: 0.014920528046786785
      policy_loss: -0.00010753957758424804
      total_loss: 586.8517456054688
      vf_explained_var: 0.9784074425697327
      vf_loss: 586.8500366210938
    sample_time_ms: 19867.417
    update_time_ms: 5.573
  iterations_since_restore: 990
  node_ip: 192.168.2.102
  num_metric_batches_dropped: 0
  pid: 7681
  policy_rewar

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 7.7/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0:	RUNNING [pid=7681], 23612 s, 995 iter, 9950000 ts, 976 rew

Result for PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0:
  custom_metrics: {}
  date: 2019-03-31_00-43-57
  done: false
  episode_len_mean: 168.25
  episode_reward_max: 1160.7607466008942
  episode_reward_mean: 1017.6002274137028
  episode_reward_min: 0.888733226812576
  episodes_this_iter: 60
  episodes_total: 58064
  experiment_id: 317c3905198e469ea331d85175f15b15
  hostname: Gandalf
  info:
    grad_time_ms: 3891.972
    load_time_ms: 1.634
    num_steps_sampled: 9960000
    num_steps_trained: 9960000
    rl_0:
      cur_kl_coeff: 0.12882612645626068
      cur_lr: 4.999999873689376e-05
      entropy: 0.33415332436561584
      kl: 0.05998612567782402

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/3 CPUs, 0/1 GPUs
Memory usage on this node: 7.7/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
TERMINATED trials:
 - PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0:	TERMINATED [pid=7681], 23732 s, 1000 iter, 10000000 ts, 952 rew

